In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import re
import pycep_correios
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
import json
import requests

Carregando o arquivo

In [114]:
# carregando dados de interesse do arquivo 
sih = pd.read_csv('sih_rj_dez19_mai20.csv', usecols=['ANO', 'MES','TIPO_ATENDIMENTO', 'CEP_PACIENTE', 'GENERO', 'CID_DESCRI', 'CID_COD','SITUACAO', 'CNES', 'RACA_COR','IDADE'])

In [115]:
# carregando os dados de cnes dos estabelecimentos
cnes = pd.read_csv('estabelecimentos2.csv', sep=';')

In [116]:
# renomeando coluna no dataframe dados para dar o merge de forma correta em cnes
sih.rename(columns={'CNES_ESTABELECIMENTO': 'CNES'}, inplace=True)

In [117]:
# visualizando o dataframe
sih.head()

,ANO,MES,TIPO_ATENDIMENTO,CEP_PACIENTE,GENERO,CID_DESCRI,CID_COD,SITUACAO,CNES,RACA_COR,IDADE
0,2020,Jan,clin_medica,21660500,M,tto hepatite viral,cid[B159]-Hepatite A sem coma hepático,internado para diagnostico,2269880,ignorado,18
1,2020,Jan,pediatria,21012140,F,tto infec. perinatal,cid[H10]-Conjuntivite,internado para diagnostico,2269880,ignorado,0
2,2020,Abr,clin_medica,21931350,F,tto infecções virais,cid[B08]-Outras infecções virais caracterizada...,internado para diagnostico,2269880,ignorado,29
3,2019,Dez,pediatria,20261004,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,2269880,parda,0
4,2019,Dez,clin_medica,21210070,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,2269880,branca,61


In [118]:
# Visualizando a estrtura do dataframe
sih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ANO               157 non-null    int64 
 1   MES               157 non-null    object
 2   TIPO_ATENDIMENTO  157 non-null    object
 3   CEP_PACIENTE      157 non-null    int64 
 4   GENERO            157 non-null    object
 5   CID_DESCRI        157 non-null    object
 6   CID_COD           157 non-null    object
 7   SITUACAO          157 non-null    object
 8   CNES              157 non-null    int64 
 9   RACA_COR          157 non-null    object
 10  IDADE             157 non-null    int64 
dtypes: int64(4), object(7)
memory usage: 13.6+ KB


In [119]:
# Identificando os cnes dos estabelecimentos
dados= pd.merge(sih, cnes, on="CNES", how="inner")


In [144]:
# visualizando o novo dataframe
dados.head()

,ANO,MES,TIPO_ATENDIMENTO,CEP_PACIENTE,GENERO,CID_DESCRI,CID_COD,SITUACAO,CNES,RACA_COR,...,RAZAO SOCIAL,CNPJ PROPRIO,CNPJ MANTENEDORA,TIPO GESTAO,LOGRADOURO,NUMERO,BAIRRO,CEP,LATITUDE,LONGITUDE
0,2020,Jan,clin_medica,21660500,M,tto hepatite viral,cid[B159]-Hepatite A sem coma hepático,internado para diagnostico,2269880,ignorado,...,MINISTERIO DA SAUDE,3.945440e+11,NaN,M,LONDRES,616,BONSUCESSO,21041000,-22.86714,-43.24854
1,2020,Jan,pediatria,21012140,F,tto infec. perinatal,cid[H10]-Conjuntivite,internado para diagnostico,2269880,ignorado,...,MINISTERIO DA SAUDE,3.945440e+11,NaN,M,LONDRES,616,BONSUCESSO,21041000,-22.86714,-43.24854
2,2020,Abr,clin_medica,21931350,F,tto infecções virais,cid[B08]-Outras infecções virais caracterizada...,internado para diagnostico,2269880,ignorado,...,MINISTERIO DA SAUDE,3.945440e+11,NaN,M,LONDRES,616,BONSUCESSO,21041000,-22.86714,-43.24854
3,2019,Dez,pediatria,20261004,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,2269880,parda,...,MINISTERIO DA SAUDE,3.945440e+11,NaN,M,LONDRES,616,BONSUCESSO,21041000,-22.86714,-43.24854
4,2019,Dez,clin_medica,21210070,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,2269880,branca,...,MINISTERIO DA SAUDE,3.945440e+11,NaN,M,LONDRES,616,BONSUCESSO,21041000,-22.86714,-43.24854


Estrturando os dados de raça

In [145]:
# Renomeando  as linhas da coluna RACA_COR considerando a observação inicial
for (row, col) in dados.iterrows():
    if col.RACA_COR == 'parda':
        dados['RACA_COR'].replace(to_replace=col.RACA_COR, value='negra', inplace=True)
    if col.RACA_COR == 'preta':
        dados['RACA_COR'].replace(to_replace=col.RACA_COR, value='negra', inplace=True)

In [149]:
dados['RACA_COR'].unique()

array(['ignorado', 'negra', 'branca', 'amarela'], dtype=object)

Identificando os endereços dos pacientes a partir do cep

In [309]:
# Funções que irão validar o cep e geolocalizar

def consulta(cep):
    cep = cep.replace('-', '')
    url = f'https://viacep.com.br/ws/{cep}/json/'
    headers = {'User-Agent': 'Autociencia/1.0'}
    resposta = requests.request('GET', url, headers=headers)
    conteudo = resposta.content.decode('utf-8')
    resposta.close()
    endereco = json.loads(conteudo)

    return endereco['cep'] + ", " + endereco['logradouro'] + ", "  + endereco['bairro'] +  ", "  + endereco['localidade']


def cep_valido(cep):
    return True if re.search(r'^(\d{5}-\d{3}|\d{8})$', cep) else False



In [152]:
# Transformando os dados de cep em string
dados['CEP_PACIENTE'] = dados['CEP_PACIENTE'].astype(str)

In [127]:
#geolocalizando os ceps
endereco = dados['CEP_PACIENTE'].apply(consulta)

In [141]:
# colocando em dicionário
df = dict(endereco)

In [128]:
#visualizando o dicionario
df

{0: '21660-500, Rua Margem da Linha, , Barros Filho, Rio de Janeiro',
 1: '21012-140, Estrada do Porto Velho, , Cordovil, Rio de Janeiro',
 2: '21931-350, Rua Gregório de Castro Morais, , Jardim Guanabara, Rio de Janeiro',
 3: '20261-004, Rua Aureliano Portugal, , Rio Comprido, Rio de Janeiro',
 4: '21210-070, Rua Arquimedes Memória, , Vila da Penha, Rio de Janeiro',
 5: '22750-003, Rua Tirol, de 982 ao fim - lado par, Freguesia (Jacarepaguá), Rio de Janeiro',
 6: '21321-220, Rua Professora Carmelita Martins, , Praça Seca, Rio de Janeiro',
 7: '21030-040, Avenida Almirante Frontin, , Ramos, Rio de Janeiro',
 8: '21341-140, Rua Maria José, , Madureira, Rio de Janeiro',
 9: '21211-210, Rua do Trabalho, , Vila da Penha, Rio de Janeiro',
 10: '20220-050, Rua Carlos Gomes, , Santo Cristo, Rio de Janeiro',
 11: '21046-405, Rua Projetada C, (Vl Pinheiro), Maré, Rio de Janeiro',
 12: '20020-030, Praça Academia, , Centro, Rio de Janeiro',
 13: '20931-480, Rua Coronel Castro Júnior, , Caju, Rio 

In [142]:
#transformando o dicionário em dataframe
df2 = pd.concat({k: pd.Series(v) for k, v in df.items()}).reset_index()

In [160]:
#excluindo colunas de não interesse
df2.drop(['level_0','level_1'], axis=1, inplace=True)

In [314]:
df2

,CEP_PACIENTE,ENDERECO_PACIENTE,COMPLEMENTO,BAIRRO_PACIENTE,LOCALIZACAO_PACIENTE
0,21660500,Rua Margem da Linha,,Barros Filho,Rio de Janeiro
1,21012140,Estrada do Porto Velho,,Cordovil,Rio de Janeiro
2,21931350,Rua Gregório de Castro Morais,,Jardim Guanabara,Rio de Janeiro
3,20261004,Rua Aureliano Portugal,,Rio Comprido,Rio de Janeiro
4,21210070,Rua Arquimedes Memória,,Vila da Penha,Rio de Janeiro
...,...,...,...,...,...
152,21235480,Rua Anhembi,,Irajá,Rio de Janeiro
153,21042020,Beco do Abacate,(Mro Timbau),Maré,Rio de Janeiro
154,21615310,Estrada São Pedro de Alcântara,até 646/647,Deodoro,Rio de Janeiro
155,20561230,Rua Alfredo Pujol,,Grajaú,Rio de Janeiro


In [313]:
#dividindo uma coluna em outras
df2[['CEP_PACIENTE','ENDERECO_PACIENTE','COMPLEMENTO','BAIRRO_PACIENTE', 'LOCALIZACAO_PACIENTE']] = df2[0].str.split(',', expand=True, n=5)

KeyError: 0

In [319]:
# Criando coluna com endereço completo
df2['END_COMPLETO_PACIENTE'] = df2['ENDERECO_PACIENTE'] + "," + df2['BAIRRO_PACIENTE']

In [156]:
#Excluindo coluna 
df2.drop([0], axis=1, inplace=True)

In [157]:
# Tirando traço dos ceps
df2['CEP_PACIENTE'] = df2['CEP_PACIENTE'].str.replace('-', '')

In [169]:
#novo dataframe com a junção dos dados geolocalizados com o restante
geoloc_dados = pd.merge(dados, df2, how="inner", on='CEP_PACIENTE')


In [171]:
geoloc_dados = geoloc_dados.drop_duplicates()

In [321]:
# adicionando ao df 
geoloc_dados['END_COMPLETO_PACIENTE'] = df2['END_COMPLETO_PACIENTE'].tolist()

In [322]:
# observando os dados
geoloc_dados['END_COMPLETO_PACIENTE']

0                     Rua Margem da Linha, Barros Filho
1                      Estrada do Porto Velho, Cordovil
2       Rua Gregório de Castro Morais, Jardim Guanabara
3                  Rua Aureliano Portugal, Rio Comprido
4                 Rua Arquimedes Memória, Vila da Penha
                             ...                       
152                                  Rua Anhembi, Irajá
153                               Beco do Abacate, Maré
154             Estrada São Pedro de Alcântara, Deodoro
155                           Rua Alfredo Pujol, Grajaú
156                      Rua Oito de Dezembro, Maracanã
Name: END_COMPLETO_PACIENTE, Length: 157, dtype: object

In [163]:
geolocator = Nominatim(user_agent="cep_pacientes_AIH_SIH")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [323]:
# Cria a coluna location com o local retornado a partir do endereço
geoloc_dados['LOCATION_PACIENTE'] = geoloc_dados['END_COMPLETO_PACIENTE'].apply(geocode)

In [351]:
# Seleciona a latitude e longitude que está dentro do local
geoloc_dados['POINT'] = geoloc_dados['LOCATION_PACIENTE'].apply(lambda loc: tuple(loc.point) if loc else None)

In [333]:
geoloc_dados['POINT'].tolist()

[(-22.8370879, -43.3680663, 0.0),
 (-22.8222284, -43.2932603, 0.0),
 (-22.8062358, -43.201497, 0.0),
 (-22.926072, -43.2155538, 0.0),
 (-22.8452131, -43.3048533, 0.0),
 (-22.9390057, -43.3277003, 0.0),
 (-22.8964799, -43.3585503, 0.0),
 None,
 (-22.8803466, -43.3449201, 0.0),
 (-22.8378272, -43.3066115, 0.0),
 (-22.9027521, -43.2031332, 0.0),
 (-22.872078, -43.2348227, 0.0),
 (-22.90994005, -43.172421852883645, 0.0),
 (-22.8785457, -43.2188006, 0.0),
 (-22.9539035, -43.176878, 0.0),
 (-22.9174708, -43.2061306, 0.0),
 (-22.8977266, -43.352237742223416, 0.0),
 (-22.8977266, -43.352237742223416, 0.0),
 (-22.9314847, -43.6507928, 0.0),
 (-22.9314847, -43.6507928, 0.0),
 None,
 (-22.9242966, -43.1827296, 0.0),
 None,
 (-22.9311419, -43.2409389, 0.0),
 (-22.979592, -43.6899749, 0.0),
 (-22.8763568, -43.3677363, 0.0),
 (-22.8990545, -43.2729055, 0.0),
 None,
 (-22.9702488, -43.1859845, 0.0),
 (-22.9034464, -43.1970419, 0.0),
 (-22.8890944, -43.2666879, 0.0),
 None,
 (-22.8274462, -43.3251559,

In [358]:
# 6 dados foram rastreados assim
geoloc_dados['POINT'].isnull().value_counts()

False    157
Name: POINT, dtype: int64

In [353]:
# Substituindo os valores nulos por (0,0,0)
geoloc_dados['POINT'].fillna("(0.0, 0.0, 0.0)", inplace=True)

In [361]:
#tirando os parenteses
geoloc_dados['POINT'] = geoloc_dados['POINT'].str.replace(r'(', '', regex=True)

In [362]:
geoloc_dados['POINT'] = geoloc_dados['POINT'].str.replace(r')', '', regex=True)

In [238]:
geoloc_dados['POINT']

0      (-22.8370879, -43.3680663, 0.0)
1      (-22.8189884, -43.2906605, 0.0)
2       (-22.8062358, -43.201497, 0.0)
3       (-22.926072, -43.2155538, 0.0)
4      (-22.8452131, -43.3048533, 0.0)
                    ...               
186     (-8.0213931, -34.9266892, 0.0)
187     (-19.945872, -44.0464402, 0.0)
188    (-22.8720617, -43.4217654, 0.0)
189     (-22.4399526, -46.823165, 0.0)
190    (-22.8191343, -43.3886026, 0.0)
Name: POINT, Length: 157, dtype: object

In [364]:
# Transformando a coluna POINT em 3 colunas: latitude, longitude e altitude
lat_long = pd.DataFrame(geoloc_dados['POINT'].str[1:-1].str.split(',', expand=True).astype(float))

In [365]:
lat_long

,0,1,2
0,22.837088,-43.368066,0.0
1,22.822228,-43.293260,0.0
2,22.806236,-43.201497,0.0
3,22.926072,-43.215554,0.0
4,22.845213,-43.304853,0.0
...,...,...,...
152,22.835920,-43.322176,0.0
153,22.863196,-43.239504,0.0
154,22.855918,-43.388443,0.0
155,22.926754,-43.263574,0.0


In [366]:
# criando colunas no dataframe do ddos gerais com lat  e long
geoloc_dados['LATITUDE_PACIENTE'] = lat_long[0]
geoloc_dados['LONGITUDE_PACIENTE'] = lat_long[1]

In [368]:
geoloc_dados.head()

,Unnamed: 0,Unnamed: 0.1,ANO,MES,TIPO_ATENDIMENTO,CEP_PACIENTE,GENERO,CID_DESCRI,CID_COD,SITUACAO,...,LONGITUDE,ENDERECO_PACIENTE,COMPLEMENTO,BAIRRO_PACIENTE,LOCALIZACAO_PACIENTE,LOCATION_PACIENTE,POINT,END_COMPLETO_PACIENTE,LATITUDE_PACIENTE,LONGITUDE_PACIENTE
0,0,0,2020,Jan,clin_medica,21660500,M,tto hepatite viral,cid[B159]-Hepatite A sem coma hepático,internado para diagnostico,...,-43.24854,Rua Margem da LinhaBarros FilhoBarros Filho,NaN,Barros Filho,Rio de Janeiro,"Rua Margem da Linha, Margem da Linha, Barros F...","-22.8370879, -43.3680663, 0.0","Rua Margem da Linha, Barros Filho",22.837088,-43.368066
1,1,1,2020,Jan,pediatria,21012140,F,tto infec. perinatal,cid[H10]-Conjuntivite,internado para diagnostico,...,-43.24854,Estrada do Porto VelhoCordovilCordovil,NaN,Cordovil,Rio de Janeiro,"Estrada do Porto Velho, Cidade Alta, Cordovil,...","-22.8222284, -43.2932603, 0.0","Estrada do Porto Velho, Cordovil",22.822228,-43.293260
2,2,2,2020,Abr,clin_medica,21931350,F,tto infecções virais,cid[B08]-Outras infecções virais caracterizada...,internado para diagnostico,...,-43.24854,Rua Gregório de Castro MoraisJardim GuanabaraJ...,NaN,Jardim Guanabara,Rio de Janeiro,"Rua Gregório de Castro Morais, Jardim Guanabar...","-22.8062358, -43.201497, 0.0","Rua Gregório de Castro Morais, Jardim Guanabara",22.806236,-43.201497
3,3,3,2019,Dez,pediatria,20261004,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,...,-43.24854,Rua Aureliano PortugalRio CompridoRio Comprido,NaN,Rio Comprido,Rio de Janeiro,"Rua Aureliano Portugal, Rio Comprido, Rio de J...","-22.926072, -43.2155538, 0.0","Rua Aureliano Portugal, Rio Comprido",22.926072,-43.215554
4,4,4,2019,Dez,clin_medica,21210070,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,...,-43.24854,Rua Arquimedes MemóriaVila da PenhaVila da Penha,NaN,Vila da Penha,Rio de Janeiro,"Rua Arquimedes Memória, Vila da Penha, Zona No...","-22.8452131, -43.3048533, 0.0","Rua Arquimedes Memória, Vila da Penha",22.845213,-43.304853


Criando uma nova coluna para zonas de localidade residente dos pacientes

In [279]:
# c riando um novo dataframe
df_zonas = pd.DataFrame()

In [378]:
df_zonas[['ENDERECO_PACIENTE','BAIRRO_PACIENTE', 'ZONA','LOCALIZACAO_PACIENTE']] = geoloc_dados['LOCATION_PACIENTE'].str.split(',', expand=True, n=3)

In [398]:
#tirando os espaços em brancos
df_zonas['ZONA'] =df_zonas['ZONA'].str.strip()

In [417]:
# Renomeando  as linhas da coluna RACA_COR considerando a observação inicial
for (row, col) in df_zonas.iterrows():
    if col.ZONA == 'Vila Divino Pai Eterno':
        df_zonas['ZONA'].replace(to_replace=col.ZONA, value='Zona Sul do Rio de Janeiro', inplace=True)
    if col.ZONA == 'Tatuapé':
        df_zonas['ZONA'].replace(to_replace=col.ZONA, value='Zona Norte do Rio de Janeiro', inplace=True)
    if col.ZONA == 'Rieuville':
        df_zonas['ZONA'].replace(to_replace=col.ZONA, value='Zona Norte do Rio de Janeiro', inplace=True)
    if col.ZONA == 'Pão de Açúcar':
        df_zonas['ZONA'].replace(to_replace=col.ZONA, value='Zona Oeste do Rio de Janeiro', inplace=True)
    if col.ZONA == 'Lucena':
        df_zonas['ZONA'].replace(to_replace=col.ZONA, value='Zona Norte do Rio de Janeiro', inplace=True)


***OBS*** *Alguns dados foram tratados no excel, pois a geolocalização gerou alguns erros*

In [440]:
# observando os dados unicos
df_zonas['ZONA'].unique()

array(['Zona Norte do Rio de Janeiro', 'Zona Central do Rio de Janeiro',
       'Zona Oeste do Rio de Janeiro', nan, 'Zona Sul do Rio de Janeiro'],
      dtype=object)

In [435]:
geoloc_dados['ZONA'] = df_zonas['ZONA'].tolist()

In [439]:
geoloc_dados['ZONA'].isnull().value_counts()

False    157
Name: ZONA, dtype: int64

In [437]:
geoloc_dados.loc[geoloc_dados['ZONA'].isnull()]

,Unnamed: 0,Unnamed: 0.1,ANO,MES,TIPO_ATENDIMENTO,CEP_PACIENTE,GENERO,CID_DESCRI,CID_COD,SITUACAO,...,ENDERECO_PACIENTE,COMPLEMENTO,BAIRRO_PACIENTE,LOCALIZACAO_PACIENTE,LOCATION_PACIENTE,POINT,END_COMPLETO_PACIENTE,LATITUDE_PACIENTE,LONGITUDE_PACIENTE,ZONA
7,7,7,2020,Fev,clin_medica,21030040,F,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,...,Avenida Almirante FrontinRamosRamos,NaN,Ramos,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Avenida Almirante Frontin, Ramos",0.0,0.0,NaN
20,20,24,2020,Fev,clin_medica,22775450,M,tto doenças bact. zoonósticas,cid[A278]-Outras formas de leptospirose,melhorado,...,Rua KazumbaTaquaraTaquara,NaN,Taquara,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Rua Kazumba, Taquara",0.0,0.0,NaN
22,22,26,2020,Mar,clin_medica,21043230,M,tto doenças bact. zoonósticas,cid[A279]-Leptospirose não especificada,administrativa,...,Rua Ivanildo AlvesBonsucessoBonsucesso,NaN,Bonsucesso,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Rua Ivanildo Alves, Bonsucesso",0.0,0.0,NaN
27,27,31,2020,Mai,clin_medica,22010060,F,tto afecções da pele,cid[B749]-Filariose não especificada,melhorado,...,Ladeira Ari BarrosoLemeLeme,NaN,Leme,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Ladeira Ari Barroso, Leme",0.0,0.0,NaN
31,31,35,2020,Fev,clin_medica,22610210,F,atend. urgência pediátrica,cid[A09]-Diarréia e gastroenterite de origem i...,melhorado,...,Estrada da CanoaSão ConradoSão Conrado,NaN,São Conrado,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Estrada da Canoa, São Conrado",0.0,0.0,NaN
42,42,50,2020,Mar,clin_medica,21044580,F,tto doenças protozoarios,cid[B572]-Doença de Chagas (crônica) com compr...,melhorado,...,Rua das RosasMaréMaré,(Nova Holanda),Maré,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Rua das Rosas, Maré",0.0,0.0,NaN
47,47,57,2020,Mar,clin_medica,22733060,F,tto doenças protozoarios,cid[B572]-Doença de Chagas (crônica) com compr...,obito com necropsia,...,Caminho da ChácaraPraça SecaPraça Seca,NaN,Praça Seca,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Caminho da Chácara, Praça Seca",0.0,0.0,NaN
52,52,64,2019,Dez,pediatria,21321790,M,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,melhorado,...,Rua São José OperárioPraça SecaPraça Seca,NaN,Praça Seca,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Rua São José Operário, Praça Seca",0.0,0.0,NaN
56,56,68,2020,Abr,clin_medica,23030370,F,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,melhorado,...,Caminho da PedreiraGuaratibaGuaratiba,NaN,Guaratiba,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Caminho da Pedreira, Guaratiba",0.0,0.0,NaN
71,71,93,2020,Mai,pediatria,21725440,F,tto doenças infec e intestinais,cid[A09]-Diarréia e gastroenterite de origem i...,internado para diagnostico,...,Rua São ValericoRealengoRealengo,NaN,Realengo,Rio de Janeiro,NaN,"0.0, 0.0, 0.0","Rua São Valerico, Realengo",0.0,0.0,NaN


In [438]:
geoloc_dados['ZONA'].fillna("Não localizado", inplace=True)

In [441]:
geoloc_dados['ZONA'].unique()

array(['Zona Norte do Rio de Janeiro', 'Zona Central do Rio de Janeiro',
       'Zona Oeste do Rio de Janeiro', 'Não localizado',
       'Zona Sul do Rio de Janeiro'], dtype=object)

In [442]:
# Salvando o arquivo
geoloc_dados.to_csv('sih_rj_dez19_mai20_tratado.csv')